# Job Upwork
- https://www.upwork.com/jobs/~01a0a84820d3d22c49

- Extrair dados de dois sites de criptomoedas
- Criar uma aba em uma planilha do gsheet pra cada tag das altcoins
- tags [ arc-20, brc-20, cbrc-20, drc-20, src-20 ]

# Entendimento do Job

- Dentre as tags mencionadas 4 delas são oriundas do site 'https://coinranking.com/', logo temos que criar um crawler para varrer o site e raspar tadas as altcoins com essas tags;

- A tag arc-20 tem suas altcoins listadas no site 'https://bitatom.io', segundo a descrição do job nesse site podemos ter um controle manual maior para obtenção dos twitters das altcoins, visto que essa informação não se encontra no site da bitatom e deveremos criar um crawler no google para gerar esses twitters.

# Tecnicas e libs usadas

- Identificação de api ocultas dispostas no site

- Estruturação de dados com dicionarios e pandas
- Raspagem de dados com bs4 e selenium
- Requisições com Requests
- Regex

# Material de apoio

- Site para converter codigo cUrl em python: https://curlconverter.com/

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
from bs4 import BeautifulSoup

In [43]:
import requests

# Header originado pelo cUrl da api
headers = {
    'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    'Accept': 'application/json, text/plain, */*',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    'sec-ch-ua-platform': '"Linux"',
}

# Foi criado um dicionario para receber todas as requisições em cada tag, dessa forma é possível iterar sobre as keys do dicionario e passar a informação da tag 
# na api e já relacionar a resposta da requisição com a tag que estamos buscando na api.
responses = {
'brc-20':[],
'cbrc-20':[],
'src-20':[],
'drc-20':[],
}


for key in responses.keys():
    # Requisição para obter o numero máx de páginas de cada tag
    r = requests.get(f'https://coinranking.com/coins/{key}')
    soup = BeautifulSoup(r.text,'html.parser')
    pagination_buttons_ammount = len(soup.select('.pagination__list a')) # verifica quantos buttons tem na paginação
    pages = int(soup.select_one(f'.pagination__list a:nth-of-type({pagination_buttons_ammount})').text) # pega o ultimo botão e armazena seu valor, já converte de str para int

    for i in range(pages): # uso o valor maximo de paginas para montar os dados da requisiçao da api
        print(f'Extraindo página {i+1} da tag {key}...')
        r = requests.get(
            f'https://coinranking.com/api/v2/coins?offset={i*50}&orderBy=marketCap&limit=50&orderDirection=desc&referenceCurrencyUuid=yhjMzLPhuIDl&timePeriod=24h&search=&tags[]={key}',
            headers=headers,# offset valor inicial de items  e limit qntd de itens que vao entrar na requisição
    )
        if r.status_code == 200:
            responses[key].append(r) # adiciona a resposta da requisição ao dicionario se sucesso
        else:
            print(f'error',r.status_code,i)
        
        print('extração concluida')
        print('-'*30)

Extraindo página 1 da tag brc-20...
extração concluida
------------------------------
Extraindo página 2 da tag brc-20...
extração concluida
------------------------------
Extraindo página 3 da tag brc-20...
extração concluida
------------------------------
Extraindo página 4 da tag brc-20...
extração concluida
------------------------------
Extraindo página 5 da tag brc-20...
extração concluida
------------------------------
Extraindo página 6 da tag brc-20...
extração concluida
------------------------------
Extraindo página 7 da tag brc-20...
extração concluida
------------------------------
Extraindo página 8 da tag brc-20...
extração concluida
------------------------------
Extraindo página 9 da tag brc-20...
extração concluida
------------------------------
Extraindo página 10 da tag brc-20...
extração concluida
------------------------------
Extraindo página 11 da tag brc-20...
extração concluida
------------------------------
Extraindo página 12 da tag brc-20...
extração conclu

In [65]:
import pandas as pd 
dfs = {} # criar um dicionario para armazenar os dataframes
for key,value in responses.items():
    
    data = { # monta a estrutura de dados do dataframe
    'token':[],
    'web':[],
    'address':[],
    'telegram':[],
    'twitter':[]
    }
    
    for r in value:
        coins = r.json().get('data').get('coins') # usa a resposta da api para pegar os dados das alt coins
        
        for coin in coins: # itera cada alt coins dentro da nossa lista de alt coins
            data['token'].append(coin.get('name'))
            data['web'].append(coin.get('coinrankingUrl'))
            data['address'].append(coin.get('contractAddresses'))
            telegram = None # vamos atribuir valor None para telegram e twitter e se acharmos esses dados posteiormente vamos substituir
            twitter = None
            
            try:
                response = requests.get(coin.get('coinrankingUrl')) # vamos fazer uma requisição para entrar em cada pagina de alt coin e rapar o twitter e telegram
                soup = BeautifulSoup(response.text,'html.parser')
                
                try:
                    links_rows = soup.select('.link-list.link-list--light tbody tr')
                    for row in links_rows:
                        if row.find('th').text.strip() == 'X (formerly Twitter)':
                            twitter = (row.find('td').text.strip().replace('\n',''))
                        elif row.find('th').text.strip() == 'Telegram':
                            telegram = (row.find('td').text.strip().replace('\n',''))
                    
                except Exception as e:
                    print(e)
            except Exception as e:
                print(e)
            
            data['telegram'].append(telegram)
            data['twitter'].append(twitter)
    
    df = pd.DataFrame(data)
    dfs[key] = df.copy(deep=True)

In [67]:
for key,value in dfs.items():
    value.to_csv(f'/home/dayvid/Documentos/Projects/webnars/07abr/data/{key}.csv',sep=';',index=False)
    

# Extração do Bitatom


In [68]:
import requests

cookies = {
    '_ga': 'GA1.1.60151747.1712353204',
    'cf_clearance': 'bsZOYc5Yt_rz5AMrNzn35lhm7Ecu0lFLvYIs702RgTI-1712407745-1.0.1.1-rKK22UgZPo3c1B.k_RpGJ1BOfetz1XvlIOsuY96e51mZ2Xt4HtGJwZp75D831v3fPqmhUqv0HVfVc3c.niG4Dg',
    '_ga_ZF7HDV7E30': 'GS1.1.1712407667.3.1.1712407773.0.0.0',
}

headers = {
    'authority': 'bitatom.io',
    'accept': '*/*',
    'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'content-type': 'application/json',
    # 'cookie': '_ga=GA1.1.60151747.1712353204; cf_clearance=bsZOYc5Yt_rz5AMrNzn35lhm7Ecu0lFLvYIs702RgTI-1712407745-1.0.1.1-rKK22UgZPo3c1B.k_RpGJ1BOfetz1XvlIOsuY96e51mZ2Xt4HtGJwZp75D831v3fPqmhUqv0HVfVc3c.niG4Dg; _ga_ZF7HDV7E30=GS1.1.1712407667.3.1.1712407773.0.0.0',
    'if-modified-since': 'Sat, 06 Apr 2024 12:49:24 GMT',
    'referer': 'https://bitatom.io/?kind=all&page=2',
    'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
}

params = {
    'batch': '1',
    'input': '{"0":{"json":{"list":"all"}},"1":{"json":null,"meta":{"values":["undefined"]}}}',
}

bita_response = requests.get(
    'https://bitatom.io/api/trpc/token.queryTokens,utils.getBitcoinPrice',
    params=params,
    cookies=cookies,
    headers=headers,
)

In [83]:
len(bita_response.json()[0].get('result').get('data').get('json'))

382

In [84]:
bita_coins = bita_response.json()[0].get('result').get('data').get('json')

In [125]:
from time import sleep
import re 

def verify_twitter(string):
    pattern = r"(\@.+)(\)\s)"
    match = re.search(pattern,string)
    if match:
        return match.group(1)
    else:
       return False

In [127]:
bita_data = {
'token':[],
'website':[],
'twitter':[]
}

for bita_coin in bita_coins:
    bita_coin = bita_coin.get('token')
    bita_data['token'].append(bita_coin.get('ticker'))
    bita_data['website'].append(f"https://bitatom.io/token/{bita_coin.get('ticker')}")
    driver.get('https://www.google.com')
    search_bar = driver.find_element(By.CLASS_NAME,'lst.Ucigb')
    search_bar.clear()
    search_bar.send_keys(f"altcoin {bita_coin.get('ticker')} twitter")
    search_bar.send_keys(Keys.ENTER)
    sleep(1)
    
    soup = BeautifulSoup(driver.page_source,'html.parser')
    titulos = soup.select('.zBAuLc.l97dzf div')
    
    titulos = [titulo.text.strip() for titulo in titulos]
    twitters = [verify_twitter(titulo) for titulo in titulos if verify_twitter(titulo)]
    print(twitters)

    bita_data['twitter'].append(','.join(twitters))

['@MotaCoin_', '@nikolasmotta', '@antoniomotamail', '@jayrmotta', '@Mota_Wallace011', '@_Hugo_Ramos_', '@thecryptovalley']
['@atombitcoin', '@cosmoshub', '@CryptoBullAtom', '@ATOMCosmonauts', '@ATOMAccelerator', '@CosmosATOMDaily']
['@pepecoineth', '@PePeCoinbsc20', '@pepenextgen', '@solanaPepecoin']
['@atombitcoin']
['@atomicalsxyz', '@Galaxylabss']
['@Shibtoken', '@ShibainuCoin', '@LucieSHIB', '@shibainuburns', '@ShytoshiKusama', '@ShibArmyBsc', '@SHIBA__BSC']
['@SATS_info', '@sats_bt', '@satoshivision75', '@Satscoinvip', '@SatsGlobal', '@Protocrypto', '@SATCoinNetwork', '@SatoshiCoinSAT']
['@buffdogecoin']
['@aBTC_Center', '@ABTC_1']
['@OrdiLaunch', '@ORDI_2_0', '@brc20vn', '@Orditoken', '@ordi_inscribe', '@ord_io', '@OrdiswapLabs']
['@atomicalsxyz', '@atomicalsmarket', '@Electron_ARC20', '@atomicalswallet', '@atomicals_punks']
['@HashAItoken', '@JjTaxin', '@HashedNetwork', '@HashDeer1', '@ZeroHashX']
['@domodata', '@domoerc20', '@cryptodomo', '@domocoin', '@CryptoDomo82', '@dominod

In [130]:
bita_df = pd.DataFrame(bita_data)
bita_df.to_csv(f'/home/dayvid/Documentos/Projects/webnars/07abr/data/arc-20.csv',sep=';',index=False)